<a href="https://colab.research.google.com/github/KolekarPramod/Financial-Forecasting-Frontier-Distributed-ML/blob/main/Financial_Forecasting_Frontier_Distributed_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Project Name**    -



##### **Project Type**    - EDA/Regression/Classification/Unsupervised
##### **Contribution**    - Individual/Team
##### **Team Member 1 -**
##### **Team Member 2 -**
##### **Team Member 3 -**
##### **Team Member 4 -**

# **Project Summary -**

Write the summary here within 500-600 words.

# **GitHub Link -**

Provide your GitHub Link here.

# **Problem Statement**


# ***Let's Begin !***

## ***1. Know Your Data***

### Import Libraries

In [ ]:
# Import Libraries
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum, countDistinct,avg

### Dataset Loading

In [ ]:
# 1. Setup Spark
import matplotlib.pyplot as plt
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("ReadCSV").getOrCreate()

In [ ]:
# 2. Load into PySpark
df = spark.read.csv('bank_data.csv', header=True, inferSchema=True)

### Dataset First View

In [ ]:
# Dataset First Look
df.show(5)

### Dataset Rows & Columns count

In [ ]:
# Dataset Rows & Columns count
df.columns

### Dataset Information

In [ ]:
# Dataset Info
df.printSchema()

#### Duplicate Values

In [ ]:
# Dataset Duplicate Value Count
df.groupBy(df.columns).count().filter("count > 1").count()

#### Missing Values/Null Values

In [ ]:
# Missing Values/Null Values Count
df.select([sum(col(c).isNull().cast("int")).alias(c) for c in df.columns]).show()

### What did you know about your dataset?

Answer Here

## ***2. Understanding Your Variables***

In [ ]:
# Dataset Describe
df.describe().show()

### Variables Description

$\color{yellow}{\text{age}}$ – Age of the individual (integer)

$\color{yellow}{\text{job}}$ – Job type (string)

$\color{yellow}{\text{marital}}$ – Marital status (string)

$\color{yellow}{\text{education}}$ – Education level (string)

$\color{yellow}{\text{default}}$ – Indicates if the individual has credit in default (string)

$\color{yellow}{\text{balance}}$ – Account balance (integer)

$\color{yellow}{\text{housing}}$ – Indicates if the individual has a housing loan (string)

$\color{yellow}{\text{loan}}$ – Indicates if the individual has a personal loan (string)

$\color{yellow}{\text{contact}}$ – Type of communication contact (string)

$\color{yellow}{\text{day}}$ – Last contact day of the month (integer)

$\color{yellow}{\text{month}}$ – Last contact month of the year (string)

$\color{yellow}{\text{duration}}$ – Last contact duration, in seconds (integer)

$\color{yellow}{\text{campaign}}$ – Number of contacts performed during this campaign for this client (integer)

$\color{yellow}{\text{pdays}}$ – Number of days since the client was last contacted from a previous campaign (integer, -1 means not contacted)

$\color{yellow}{\text{previous}}$ – Number of contacts before this campaign for the client (integer)

$\color{yellow}{\text{poutcome}}$ – Outcome of the previous marketing campaign (string)

$\color{yellow}{\text{y}}$ – Indicates if the client subscribed to a term deposit (string)

### Check Unique Values for each variable.

In [ ]:
# Check Unique Values for each variable.
df.select([countDistinct(c).alias(c) for c in df.columns]).show()

## 3. ***Data Wrangling***

### Data Wrangling Code

In [ ]:
# Write your code to make your dataset analysis ready.

### What all manipulations have you done and insights you found?

Answer Here.

## ***4. Data Vizualization, Storytelling & Experimenting with charts : Understand the relationships between variables***

#### Chart - 1

In [ ]:
# Initialize a Spark session
spark = SparkSession.builder.appName("EDA with PySpark").getOrCreate()

In [ ]:
# Step 1: Get value counts from Spark
vc_df = df.groupBy("y").count().orderBy("count", ascending=False)

# Step 2: Convert to pandas (small data only)
vc_pd = vc_df.toPandas()

# Step 3: Plot bar chart
plt.figure(figsize=(6, 4))
plt.bar(vc_pd["y"], vc_pd["count"], color='skyblue')
plt.xlabel("Target: y (Subscribed)")
plt.ylabel("Count")
plt.title("Term Deposit Subscription Count (Yes/No)")
plt.grid(axis='y')
plt.show()


##### 1. Why did you pick the specific chart?

Answer Here.

##### 2. What is/are the insight(s) found from the chart?

Answer Here

##### 3. Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

Answer Here

#### Chart - 2

In [ ]:
# Chart - 2 visualization code

# Step 1: Get counts
job_counts = df.groupBy("job").count().orderBy("count", ascending=False)

# Step 2: Convert to pandas
job_pd = job_counts.toPandas()

# Step 3: Plot bar chart
plt.figure(figsize=(10, 5))
plt.bar(job_pd["job"], job_pd["count"], color='orange')
plt.xlabel("Job Type")
plt.ylabel("Number of Clients")
plt.title("Count of Clients by Job Type")
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.grid(axis='y')
plt.show()

##### 1. Why did you pick the specific chart?

Answer Here.

##### 2. What is/are the insight(s) found from the chart?

Answer Here

##### 3. Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

Answer Here

#### Chart - 3

In [ ]:
# Chart - 3 visualization code
# Step 1: Group and count by education + target
edu_counts = df.groupBy("education", "y").count().orderBy("education", "y")

# Step 2: Pivot to wide format
edu_pivot = edu_counts.groupBy("education").pivot("y").sum("count").fillna(0)

# Step 3: Convert to pandas and plot
edu_pd = edu_pivot.toPandas()
edu_pd["total"] = edu_pd["yes"] + edu_pd["no"]
edu_pd["subscription_rate"] = edu_pd["yes"] / edu_pd["total"]

# Step 4: Plot
import matplotlib.pyplot as plt

plt.figure(figsize=(8, 4))
plt.bar(edu_pd["education"], edu_pd["subscription_rate"], color="green")
plt.xlabel("Education Level")
plt.ylabel("Subscription Rate")
plt.title("Subscription Rate by Education Level")
plt.xticks(rotation=45)
plt.grid(axis='y')
plt.tight_layout()
plt.show()

##### 1. Why did you pick the specific chart?

Answer Here.

##### 2. What is/are the insight(s) found from the chart?

Answer Here

##### 3. Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

Answer Here

#### Chart - 4

In [ ]:
# Chart - 4 visualization code
# Group and count by housing status and y
house_counts = df.groupBy("housing", "y").count().orderBy("housing", "y")

# Pivot
house_pivot = house_counts.groupBy("housing").pivot("y").sum("count").fillna(0)

# To pandas
house_pd = house_pivot.toPandas()
house_pd["total"] = house_pd["yes"] + house_pd["no"]
house_pd["subscription_rate"] = house_pd["yes"] / house_pd["total"]

# Plot
plt.figure(figsize=(6, 4))
plt.bar(house_pd["housing"], house_pd["subscription_rate"], color="purple")
plt.xlabel("Housing Loan")
plt.ylabel("Subscription Rate")
plt.title("Subscription Rate by Housing Loan")
plt.grid(axis='y')
plt.tight_layout()
plt.show()


##### 1. Why did you pick the specific chart?

Answer Here.

##### 2. What is/are the insight(s) found from the chart?

Answer Here

##### 3. Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

Answer Here

#### Chart - 5

In [ ]:
# Chart - 5 visualization code
# Compute average balance by 'y'
avg_balance = df.groupBy("y").agg(avg("balance").alias("avg_balance")).orderBy("y")

# To pandas
avg_balance_pd = avg_balance.toPandas()

# Plot
plt.figure(figsize=(6, 4))
plt.bar(avg_balance_pd["y"], avg_balance_pd["avg_balance"], color="teal")
plt.xlabel("Subscription Status")
plt.ylabel("Average Account Balance")
plt.title("Average Balance vs Subscription")
plt.grid(axis='y')
plt.tight_layout()
plt.show()


##### 1. Why did you pick the specific chart?

Answer Here.

##### 2. What is/are the insight(s) found from the chart?

Answer Here

##### 3. Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

Answer Here

#### Chart - 6

In [ ]:
# Chart - 6 visualization code

##### 1. Why did you pick the specific chart?

Answer Here.

##### 2. What is/are the insight(s) found from the chart?

Answer Here

##### 3. Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

Answer Here

#### Chart - 7

In [ ]:
# Chart - 7 visualization code

##### 1. Why did you pick the specific chart?

Answer Here.

##### 2. What is/are the insight(s) found from the chart?

Answer Here

##### 3. Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

Answer Here

#### Chart - 8

In [ ]:
# Chart - 8 visualization code

##### 1. Why did you pick the specific chart?

Answer Here.

##### 2. What is/are the insight(s) found from the chart?

Answer Here

##### 3. Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

Answer Here

#### Chart - 9

In [ ]:
# Chart - 9 visualization code

##### 1. Why did you pick the specific chart?

Answer Here.

##### 2. What is/are the insight(s) found from the chart?

Answer Here

##### 3. Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

Answer Here

#### Chart - 10

In [ ]:
# Chart - 10 visualization code

##### 1. Why did you pick the specific chart?

Answer Here.

##### 2. What is/are the insight(s) found from the chart?

Answer Here

##### 3. Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

Answer Here

#### Chart - 11

In [ ]:
# Chart - 11 visualization code

##### 1. Why did you pick the specific chart?

Answer Here.

##### 2. What is/are the insight(s) found from the chart?

Answer Here

##### 3. Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

Answer Here

#### Chart - 12

In [ ]:
# Chart - 12 visualization code

##### 1. Why did you pick the specific chart?

Answer Here.

##### 2. What is/are the insight(s) found from the chart?

Answer Here

##### 3. Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

Answer Here

#### Chart - 13

In [ ]:
# Chart - 13 visualization code

##### 1. Why did you pick the specific chart?

Answer Here.

##### 2. What is/are the insight(s) found from the chart?

Answer Here

##### 3. Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

Answer Here

#### Chart - 14 - Correlation Heatmap

In [ ]:
# Correlation Heatmap visualization code

##### 1. Why did you pick the specific chart?

Answer Here.

##### 2. What is/are the insight(s) found from the chart?

Answer Here

#### Chart - 15 - Pair Plot

In [ ]:
# Pair Plot visualization code

Answer Here

## ***7. ML Model Implementation***

In [ ]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler, StringIndexer
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# Create Spark session
spark = SparkSession.builder \
    .appName("BankTermDepositPrediction") \
    .config("spark.sql.shuffle.partitions", "4") \
    .config("spark.executor.memory", "2g") \
    .config("spark.driver.memory", "2g") \
    .getOrCreate()

# Load the dataset
data_path = "/content/bank_data.csv"
data = spark.read.csv(data_path, header=True, inferSchema=True)
# data = df
# Handle categorical variables using StringIndexer
categorical_columns = ["job", "marital", "education", "default", "housing", "loan", "contact", "month", "poutcome"]
indexers = [StringIndexer(inputCol=col, outputCol=col+"_index") for col in categorical_columns]

# Rename the target column to 'label'
data = data.withColumnRenamed("y", "label")

# Convert the label column to numerical values
label_indexer = StringIndexer(inputCol="label", outputCol="label_index")
data = label_indexer.fit(data).transform(data)

# Assemble features into a single vector
assembler = VectorAssembler(
    inputCols=["age", "balance", "day", "duration", "campaign", "pdays", "previous"] + [col+"_index" for col in categorical_columns],
    outputCol="features"
)

# Initialize the Random Forest classifier
rf = RandomForestClassifier(labelCol="label_index", featuresCol="features", numTrees=100)

# Define the pipeline
pipeline = Pipeline(stages=indexers + [assembler, rf])

# Split the data into training and test sets
train_data, test_data = data.randomSplit([0.8, 0.2], seed=1234)

# Train the model
model = pipeline.fit(train_data)

# Evaluate the model
predictions = model.transform(test_data)
evaluator = BinaryClassificationEvaluator(labelCol="label_index", metricName="areaUnderROC")
auc = evaluator.evaluate(predictions)
print(f"Test AUC: {auc}")


# Save the trained model, overwriting if the path already exists
model.write().overwrite().save("/content/ml_model_trained")

# Stop the Spark session
spark.stop()


### ML Model - 1

In [ ]:
# ML Model - 1 Implementation

# Fit the Algorithm

# Predict on the model

#### 1. Explain the ML Model used and it's performance using Evaluation metric Score Chart.

In [ ]:
# Visualizing evaluation Metric Score chart

#### 2. Cross- Validation & Hyperparameter Tuning

In [ ]:
# ML Model - 1 Implementation with hyperparameter optimization techniques (i.e., GridSearch CV, RandomSearch CV, Bayesian Optimization etc.)

# Fit the Algorithm

# Predict on the model

##### Which hyperparameter optimization technique have you used and why?

Answer Here.

##### Have you seen any improvement? Note down the improvement with updates Evaluation metric Score Chart.

Answer Here.

### ML Model - 2

#### 1. Explain the ML Model used and it's performance using Evaluation metric Score Chart.

In [ ]:
# Visualizing evaluation Metric Score chart

#### 2. Cross- Validation & Hyperparameter Tuning

In [ ]:
# ML Model - 1 Implementation with hyperparameter optimization techniques (i.e., GridSearch CV, RandomSearch CV, Bayesian Optimization etc.)

# Fit the Algorithm

# Predict on the model

##### Which hyperparameter optimization technique have you used and why?

Answer Here.

##### Have you seen any improvement? Note down the improvement with updates Evaluation metric Score Chart.

Answer Here.

#### 3. Explain each evaluation metric's indication towards business and the business impact pf the ML model used.

Answer Here.

### ML Model - 3

In [ ]:
# ML Model - 3 Implementation

# Fit the Algorithm

# Predict on the model

#### 1. Explain the ML Model used and it's performance using Evaluation metric Score Chart.

In [ ]:
# Visualizing evaluation Metric Score chart

#### 2. Cross- Validation & Hyperparameter Tuning

In [ ]:
# ML Model - 3 Implementation with hyperparameter optimization techniques (i.e., GridSearch CV, RandomSearch CV, Bayesian Optimization etc.)

# Fit the Algorithm

# Predict on the model

##### Which hyperparameter optimization technique have you used and why?

Answer Here.

##### Have you seen any improvement? Note down the improvement with updates Evaluation metric Score Chart.

Answer Here.

### 1. Which Evaluation metrics did you consider for a positive business impact and why?

Answer Here.

### 2. Which ML model did you choose from the above created models as your final prediction model and why?

Answer Here.

### 3. Explain the model which you have used and the feature importance using any model explainability tool?

Answer Here.

## ***8.*** ***Future Work (Optional)***

### 1. Save the best performing ml model in a pickle file or joblib file format for deployment process.


In [ ]:
# Save the File

### 2. Again Load the saved model file and try to predict unseen data for a sanity check.


In [ ]:
# Load the File and predict unseen data.

### ***Congrats! Your model is successfully created and ready for deployment on a live server for a real user interaction !!!***

# **Conclusion**

Write the conclusion here.

### ***Hurrah! You have successfully completed your Machine Learning Capstone Project !!!***